In [1]:
import os
import numpy as np
import pandas as pd

import sys
sys.path.append('/mnt/disk1/yunseob/Pytorch/1_CapsuleEndo/algorithms')
# from ce_utils.record import printProgress, Sec_to_M_S_ms
# from ce_utils.video import load_frame, find_clip_idxs, merging, captioning
# from ce_utils.data import batch_idxs, Reshape4torch
# from ce_model.CNN_inference import CNN_model
from ce_utils.video import load_frames, model_pred, find_clip_idxs, get_gradcam, process_and_save
from ce_utils.record import sec_to_m_s_ms, total_sum_time
from ce_model.cnn_inf import cnn_model

In [2]:
model_dir = '/mnt/disk1/yunseob/Pytorch/1_CapsuleEndo/model_developlment/1_detection_localization/2_data_aug/model/'

gpu_idx = 0

B_000_model = cnn_model(network = 'CNN_v1', n_ch = 3, n_cls = 2, 
                       model_dir = model_dir, model_name = 'B_---_0', gpu_idx = gpu_idx)

B_100_model = cnn_model(network = 'CNN_v1', n_ch = 3, n_cls = 2, 
                       model_dir = model_dir, model_name = 'B_f--', gpu_idx = gpu_idx)

B_010_model = cnn_model(network = 'CNN_v1', n_ch = 3, n_cls = 2, 
                       model_dir = model_dir, model_name = 'B_-r-', gpu_idx = gpu_idx)

B_001_model = cnn_model(network = 'CNN_v1', n_ch = 3, n_cls = 2, 
                       model_dir = model_dir, model_name = 'B_--b', gpu_idx = gpu_idx)

B_110_model = cnn_model(network = 'CNN_v1', n_ch = 3, n_cls = 2, 
                       model_dir = model_dir, model_name = 'B_fr-', gpu_idx = gpu_idx)

B_111_model = cnn_model(network = 'CNN_v1', n_ch = 3, n_cls = 2, 
                       model_dir = model_dir, model_name = 'B_frb_0', gpu_idx = gpu_idx)

B_111_sv_model = cnn_model(network = 'CNN_v1', n_ch = 3, n_cls = 2, 
                       model_dir = model_dir, model_name = 'B_frb_sv', gpu_idx = gpu_idx)

model_dir = '/mnt/disk1/yunseob/Pytorch/1_CapsuleEndo/model_developlment/2_AI_feedback/2_add_200713_1/model/'

np_200713_1_model = cnn_model(network = 'CNN_v1', n_ch = 3, n_cls = 2, 
                              model_dir = model_dir, model_name = 'np_binary', gpu_idx = gpu_idx)

model_dir = '/mnt/disk1/yunseob/Pytorch/1_CapsuleEndo/model_developlment/2_AI_feedback/3_add_200713_2/re-learning_200713_1_model/model/'

np_200713_2_model = cnn_model(network = 'CNN_v1', n_ch = 3, n_cls = 2, 
                              model_dir = model_dir, model_name = 'np_binary', gpu_idx = gpu_idx)


model_dir = '/mnt/disk1/yunseob/Pytorch/1_CapsuleEndo/model_developlment/2_AI_feedback/4_add_200917/re-learning_200713_2_model/model/'

np_200917_model = cnn_model(network = 'CNN_v1', n_ch = 3, n_cls = 2, 
                              model_dir = model_dir, model_name = 'np_binary', gpu_idx = gpu_idx)

model: B_---_0.0001_16_2007142232_069_t_accr_0.9656_t_loss_0.347245_v_accr_0.9503_v_loss_0.364293.pt 

model: B_f--_0.0001_8_2007151017_037_t_accr_0.9627_t_loss_0.350014_v_accr_0.9537_v_loss_0.357257.pt 

model: B_-r-_0.0001_8_2007161324_046_t_accr_0.9800_t_loss_0.333166_v_accr_0.9740_v_loss_0.339054.pt 

model: B_--b_0.0001_8_2007170631_037_t_accr_0.9913_t_loss_0.322011_v_accr_0.9571_v_loss_0.356080.pt 

model: B_fr-_0.0001_16_2007190546_044_t_accr_0.9783_t_loss_0.334562_v_accr_0.9672_v_loss_0.346381.pt 

model: B_frb_0.0001_32_2007230941_010_t_accr_0.9827_t_loss_0.329607_v_accr_0.9751_v_loss_0.336306.pt 

model: B_frb_sv_0.0001_32_2007290554_016_t_accr_0.9961_t_loss_0.317016_v_accr_0.9774_v_loss_0.334313.pt 

model: np_binary_0.0001_8_2009101037_199_t_accr_0.9819_t_loss_0.331068_v_accr_0.9400_v_loss_0.373449.pt 

model: np_binary_0.0001_32_2010141023_117_t_accr_0.9772_t_loss_0.335948_v_accr_0.9340_v_loss_0.377869.pt 

model: np_binary_0.0001_16_2010162056_010_t_accr_0.9627_t_loss_0.3

In [3]:
model_list = [B_000_model, B_100_model, B_010_model, B_001_model,
              B_110_model, B_111_model, B_111_sv_model, 
              np_200713_1_model, np_200713_2_model, np_200917_model]

model_ids = ['-', 'flip (r)', 'rotate (r)', 'blur (b)', 'f + r', 
              'f + r + b', 'f + r + b + sv', '200713_1 + r', '200713_2 + r', '200917 + r']

In [28]:
frame_root = '/mnt/disk2/data/private_data/SMhospital/capsule/0 data/video/200204 clip for clinical test'
video_log = pd.read_csv(frame_root + '/readme.txt', header = None, delimiter = '|', encoding = "cp949")

screening_rate = pd.DataFrame(columns = ['model'] + [i for i in range(1, len(video_log) + 1)])

screening_rate['model'] = model_ids
screening_rate = screening_rate.set_index('model')

print(video_log, '\n')
print('load frames...\n')

video_frames = []
for i in range(len(video_log)):
    
    video_info = video_log.iloc[i]
        
    idx, video_name, frame_save_time = (video_info[0],  video_info[1], video_info[2])
    print(idx, ':', video_name, '\n')

    frame_dir = os.path.join(frame_root, '{:02d}'.format(idx))

    # 프레임을 numpy 형태로 불러옴 & 걸린 시간도 가져옴
    frames_pre, frame_load_time = load_frames(frame_dir)
    print('load time:', frame_load_time, '\n')
    
    n_frame = len(frames_pre)
    
    print('model inference, processing, and saving... \n')
    for m, (model, model_id) in enumerate(zip(model_list, model_ids)):
        
        
        # Binary Model과 이미지 파일을 model_pred_and_time 함수에 넣고 리턴으로 binary model의 예측 결과와 걸린 시간을 가져옴
        # 예측결과는 model의 softmax 결과 값이 담긴 list
        preds, pred_time = model_pred(model, frames_pre)
        
        # 각각의 예측 모델이 예측한 frame은 softmax의 argmax 결과가 1인 index에 해당하는 frame
        # prediction에 해당하는 index만 가져옴 
        pred_idx = np.where(np.argmax(preds, axis = 1) == 1)[0]
        
        screening_rate.loc[model_id][idx] = '{:0.2f} %'.format(len(pred_idx)/n_frame*100)
        
    print()
    
screening_rate

     0                                      1           2
0    1       김 경숙 (03319309) 05 Aug 19_1.mpg    02:07.xxx
1    2       나 기용 (17816122) 29 Jul 19_1.mpg    01:16.xxx
2    3       모 금녀 (16548422) 10 Aug 18_1.mpg    06:19.xxx
3    4       박 완웅 (11123900) 05 Oct 18_1.mpg    01:19.xxx
4    5       송 복순 (08952031) 20 Dec 17_1.mpg    02:47.xxx
5    6   시로츠카 아키요 (20174705) 31 Jul 19_1.mpg    02:44.xxx
6    7       신 용철 (03526121) 17 Aug 18_1.mpg    07:21.xxx
7    8       이 송준 (12548633) 25 Jun 19_1.mpg    07:44.xxx
8    9       이 순천 (07422665) 18 Mar 19_1.mpg    04:17.xxx
9   10       이 재일 (05629743) 07 Feb 19_1.mpg    04:49.xxx
10  11       정 병국 (09609741) 27 Aug 18_1.mpg    01:57.xxx
11  12       정 선자 (13757465) 29 Jun 18_1.mpg    03:16.xxx
12  13       차 난희 (07808670) 03 Jul 18_1.mpg    02:19.xxx
13  14       최 금주 (07481795) 08 May 18_1.mpg    04:48.xxx
14  15       한 봉이 (20009008) 01 Jul 19_1.mpg    01:48.xxx 

load frames...

1 :  김 경숙 (03319309) 05 Aug 19_1.mpg  

6358 |########

7551 |################################################################################ | 100.0% loading frames
load time: 01:36.674 

model inference, processing, and saving... 

7551: 472(*16) |################################################## | 100.0% prediction
7551: 472(*16) |################################################## | 100.0% prediction
7551: 472(*16) |################################################## | 100.0% prediction
7551: 472(*16) |################################################## | 100.0% prediction
7551: 472(*16) |################################################## | 100.0% prediction
7551: 472(*16) |################################################## | 100.0% prediction
7551: 472(*16) |################################################## | 100.0% prediction
7551: 472(*16) |################################################## | 100.0% prediction
7551: 472(*16) |################################################## | 100.0% prediction
7551: 472(*16) |######################

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
model,,,,,,,,,,,,,,,
-,6.70 %,24.39 %,31.01 %,18.92 %,7.75 %,24.00 %,20.91 %,25.28 %,19.23 %,27.34 %,35.45 %,23.72 %,15.77 %,9.11 %,41.88 %
flip (r),3.24 %,5.32 %,11.98 %,6.94 %,1.46 %,6.57 %,12.97 %,9.12 %,4.09 %,18.37 %,9.52 %,5.01 %,2.47 %,5.11 %,29.10 %
rotate (r),6.21 %,23.27 %,26.35 %,10.72 %,5.98 %,12.94 %,14.66 %,9.76 %,11.79 %,18.29 %,16.60 %,10.90 %,5.02 %,10.38 %,28.70 %
blur (b),5.44 %,7.62 %,10.46 %,8.30 %,3.26 %,7.97 %,11.43 %,10.93 %,10.45 %,20.76 %,12.63 %,10.56 %,5.60 %,9.40 %,25.18 %
f + r,5.80 %,23.66 %,16.53 %,12.73 %,4.07 %,12.46 %,19.66 %,12.10 %,9.09 %,23.71 %,20.73 %,12.75 %,6.35 %,7.80 %,31.84 %
f + r + b,4.86 %,19.46 %,16.21 %,9.72 %,4.91 %,14.98 %,21.87 %,11.75 %,16.69 %,12.43 %,15.21 %,12.48 %,5.81 %,9.46 %,29.04 %
f + r + b + sv,6.48 %,23.22 %,29.29 %,9.07 %,4.80 %,16.14 %,15.73 %,10.46 %,21.06 %,16.14 %,26.37 %,18.38 %,5.27 %,9.05 %,29.84 %
200713_1 + r,4.73 %,16.45 %,10.41 %,8.71 %,2.40 %,10.94 %,15.79 %,9.91 %,7.22 %,22.61 %,12.84 %,9.15 %,5.87 %,6.37 %,23.49 %
200713_2 + r,5.91 %,9.60 %,11.01 %,7.53 %,4.40 %,6.80 %,11.06 %,4.50 %,5.20 %,13.42 %,4.10 %,7.27 %,6.32 %,4.90 %,12.24 %


In [31]:
screening_rate

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
model,,,,,,,,,,,,,,,
-,6.70 %,24.39 %,31.01 %,18.92 %,7.75 %,24.00 %,20.91 %,25.28 %,19.23 %,27.34 %,35.45 %,23.72 %,15.77 %,9.11 %,41.88 %
flip (r),3.24 %,5.32 %,11.98 %,6.94 %,1.46 %,6.57 %,12.97 %,9.12 %,4.09 %,18.37 %,9.52 %,5.01 %,2.47 %,5.11 %,29.10 %
rotate (r),6.21 %,23.27 %,26.35 %,10.72 %,5.98 %,12.94 %,14.66 %,9.76 %,11.79 %,18.29 %,16.60 %,10.90 %,5.02 %,10.38 %,28.70 %
blur (b),5.44 %,7.62 %,10.46 %,8.30 %,3.26 %,7.97 %,11.43 %,10.93 %,10.45 %,20.76 %,12.63 %,10.56 %,5.60 %,9.40 %,25.18 %
f + r,5.80 %,23.66 %,16.53 %,12.73 %,4.07 %,12.46 %,19.66 %,12.10 %,9.09 %,23.71 %,20.73 %,12.75 %,6.35 %,7.80 %,31.84 %
f + r + b,4.86 %,19.46 %,16.21 %,9.72 %,4.91 %,14.98 %,21.87 %,11.75 %,16.69 %,12.43 %,15.21 %,12.48 %,5.81 %,9.46 %,29.04 %
f + r + b + sv,6.48 %,23.22 %,29.29 %,9.07 %,4.80 %,16.14 %,15.73 %,10.46 %,21.06 %,16.14 %,26.37 %,18.38 %,5.27 %,9.05 %,29.84 %
200713_1 + r,4.73 %,16.45 %,10.41 %,8.71 %,2.40 %,10.94 %,15.79 %,9.91 %,7.22 %,22.61 %,12.84 %,9.15 %,5.87 %,6.37 %,23.49 %
200713_2 + r,5.91 %,9.60 %,11.01 %,7.53 %,4.40 %,6.80 %,11.06 %,4.50 %,5.20 %,13.42 %,4.10 %,7.27 %,6.32 %,4.90 %,12.24 %


In [34]:
screening_rate = screening_rate.reset_index()
screening_rate = screening_rate.set_index(pd.Index([i for i in range(1, len(model_list)+1)]))
screening_rate

,model,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,-,6.70 %,24.39 %,31.01 %,18.92 %,7.75 %,24.00 %,20.91 %,25.28 %,19.23 %,27.34 %,35.45 %,23.72 %,15.77 %,9.11 %,41.88 %
2,flip (r),3.24 %,5.32 %,11.98 %,6.94 %,1.46 %,6.57 %,12.97 %,9.12 %,4.09 %,18.37 %,9.52 %,5.01 %,2.47 %,5.11 %,29.10 %
3,rotate (r),6.21 %,23.27 %,26.35 %,10.72 %,5.98 %,12.94 %,14.66 %,9.76 %,11.79 %,18.29 %,16.60 %,10.90 %,5.02 %,10.38 %,28.70 %
4,blur (b),5.44 %,7.62 %,10.46 %,8.30 %,3.26 %,7.97 %,11.43 %,10.93 %,10.45 %,20.76 %,12.63 %,10.56 %,5.60 %,9.40 %,25.18 %
5,f + r,5.80 %,23.66 %,16.53 %,12.73 %,4.07 %,12.46 %,19.66 %,12.10 %,9.09 %,23.71 %,20.73 %,12.75 %,6.35 %,7.80 %,31.84 %
6,f + r + b,4.86 %,19.46 %,16.21 %,9.72 %,4.91 %,14.98 %,21.87 %,11.75 %,16.69 %,12.43 %,15.21 %,12.48 %,5.81 %,9.46 %,29.04 %
7,f + r + b + sv,6.48 %,23.22 %,29.29 %,9.07 %,4.80 %,16.14 %,15.73 %,10.46 %,21.06 %,16.14 %,26.37 %,18.38 %,5.27 %,9.05 %,29.84 %
8,200713_1 + r,4.73 %,16.45 %,10.41 %,8.71 %,2.40 %,10.94 %,15.79 %,9.91 %,7.22 %,22.61 %,12.84 %,9.15 %,5.87 %,6.37 %,23.49 %
9,200713_2 + r,5.91 %,9.60 %,11.01 %,7.53 %,4.40 %,6.80 %,11.06 %,4.50 %,5.20 %,13.42 %,4.10 %,7.27 %,6.32 %,4.90 %,12.24 %
10,200917 + r,3.07 %,3.68 %,4.91 %,3.39 %,1.00 %,1.15 %,2.06 %,0.72 %,1.76 %,1.25 %,0.94 %,2.17 %,1.23 %,3.15 %,8.33 %


In [35]:
screening_rate.to_csv('screening rate.csv',encoding='utf-8-sig', index = True)